Welcome to Pupper lovers!! by:  Taylor Kramer, Manisha Jaiswal, and John Santiago


In [1]:
import math
import sklearn
from sklearn import preprocessing
import pandas as pd
import torch
import tensorflow as tf
from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt

import cv2
import urllib.request

ModuleNotFoundError: No module named 'torch'

In [ ]:
#Preparing datasets

# !!!!!!!!!! when making full dataset: this took me 50 minutes to run, so conservatively expect it to take around an hour!!!!!!!!!
#(once this step is done avoid rerunning this every time since it will take a while to create the datasets every time)

def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

trainurl = 'https://github.com/johnmburt/projects/blob/master/AWS/sagemaker_dog_breed_id/dog_breeds_all_fold_1_train.lst?raw=true'
testurl = 'https://github.com/johnmburt/projects/blob/master/AWS/sagemaker_dog_breed_id/dog_breeds_all_fold_1_test.lst?raw=true'
traindf = pd.read_csv(trainurl, sep='\t', usecols=[2])
testdf = pd.read_csv(testurl, sep='\t', usecols=[2])
trainlist = traindf.values.tolist()
testlist = testdf.values.tolist()

#choose how much of the dataset to use (great for when we are still writing the code for the model)

percentdatasetused = 0.06 # use this to change dataset sizes for when we aren't running full model!!!!!!!!!!!!!!!!!!!!!
trainnum = int(len(trainlist)*percentdatasetused)
testnum = int(len(testlist)*percentdatasetused)
print(trainnum, testnum)

X_train = []
y_train = []
X_test = []
y_test = []

cnt = 0
for i in range(trainnum):
  string = trainlist[i][0]
  url = "http://vision.stanford.edu/aditya86/ImageNetDogs/thumbnails/"+string
  img = url_to_image(url)
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  gray = np.float32(gray)
  X_train.append(gray)
  label = string[string.find("-") + 1: string.find("/")]
  y_train.append(label)
  cnt = cnt + 1
  print(cnt)
cnt = 0
for j in range(testnum):
  string = testlist[j][0]
  url = "http://vision.stanford.edu/aditya86/ImageNetDogs/thumbnails/"+string
  img = url_to_image(url)
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  gray = np.float32(gray)
  X_test.append(gray)
  label = string[string.find("-") + 1: string.find("/")]
  y_test.append(label)
  cnt = cnt + 1
  print(cnt)

In [ ]:
# print(len(X_train),len(y_train))
# print(len(X_test),len(y_test))

settrain = set(y_train)
settest = set(y_test)
uniqtrain = len(settrain)
uniqtest = len(settest)
print("we have data for",str(uniqtrain),"/120 different dog breeds in train set.")
print("we have data for",str(uniqtest),"/120 different dog breeds in test set.")

# turn dog breeds into numbers where each dog class is identified as an integer
le = preprocessing.LabelEncoder()
le.fit(list(settrain))
traintargets = le.transform(y_train)
testtargets = le.transform(y_test)

we have data for 120 /120 different dog breeds in train set.
we have data for 107 /120 different dog breeds in test set.


In [ ]:
#make data insertable into model
Xtrain = tf.convert_to_tensor(X_train)
ytrain = tf.convert_to_tensor(traintargets)
Xtest = tf.convert_to_tensor(X_test)
ytest = tf.convert_to_tensor(testtargets)

In [ ]:
#create model (below is an example from one of our assignments)

simple_layers = [
                 tf.keras.layers.Flatten(),
                 tf.keras.layers.Dense(105, activation="relu"),
                 tf.keras.layers.Dropout(0.04),
                 tf.keras.layers.Dense(210, activation="relu"),
                 tf.keras.layers.Dropout(0.0035),
                 tf.keras.layers.Dense(120),
]

simple_model = Sequential(simple_layers)
simple_model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#train model
simple_model.fit(Xtrain, ytrain, epochs=5)
simple_model.summary()


#TODO: test model


Epoch 1/5
31/31 [==============================] - 1s 17ms/step - loss: 13.5565 - accuracy: 0.0072
Epoch 2/5
31/31 [==============================] - 1s 17ms/step - loss: 14.3573 - accuracy: 0.0069
Epoch 3/5
31/31 [==============================] - 1s 17ms/step - loss: 14.5315 - accuracy: 0.0138
Epoch 4/5
31/31 [==============================] - 1s 17ms/step - loss: 14.7683 - accuracy: 0.0122
Epoch 5/5
31/31 [==============================] - 1s 17ms/step - loss: 14.5695 - accuracy: 0.0106
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_27 (Dense)             (None, 105)               1720425   
_________________________________________________________________
dropout_18 (Dropout)         (None, 105)               0         
_____________________

In [ ]:
#TODO: show results in some fun way (maybe use brand new dog picture from online)

#maybe we can test our model using a brand new photo that the model hasn't seen yet? using .predict()

In [1]:
#references:

#dataset refs

'''
Primary:
  Aditya Khosla, Nityananda Jayadevaprakash, Bangpeng Yao and Li Fei-Fei. Novel dataset for Fine-Grained Image Categorization. 
  First Workshop on Fine-Grained Visual Categorization (FGVC), IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 
  2011.  [pdf]  [poster]  [BibTex]

Secondary:
  J. Deng, W. Dong, R. Socher, L.-J. Li, K. Li and L. Fei-Fei, ImageNet: A Large-Scale Hierarchical Image Database. 
  IEEE Computer Vision and Pattern Recognition (CVPR), 2009.  [pdf]  [BibTex]
'''

'\nPrimary:\n  Aditya Khosla, Nityananda Jayadevaprakash, Bangpeng Yao and Li Fei-Fei. Novel dataset for Fine-Grained Image Categorization. \n  First Workshop on Fine-Grained Visual Categorization (FGVC), IEEE Conference on Computer Vision and Pattern Recognition (CVPR), \n  2011.  [pdf]  [poster]  [BibTex]\n\nSecondary:\n  J. Deng, W. Dong, R. Socher, L.-J. Li, K. Li and L. Fei-Fei, ImageNet: A Large-Scale Hierarchical Image Database. \n  IEEE Computer Vision and Pattern Recognition (CVPR), 2009.  [pdf]  [BibTex]\n'